In [1]:
from collections import namedtuple
from decimal import Decimal as D
import numpy as np
import scipy as sp
import datetime

In [2]:
# index into the price tree
# t is total number of months, n_up is number of "up" months
class I(namedtuple('IndexBase', 't n_up')):
    def __init__(self, t, n_up):
        assert 0 <= self.n_up <= self.t, \
            "We want 0 <= %s <= %s" % (self.n_up, self.t)

    @property
    def n_down(self):
        return self.t - self.n_up

    def go_up(self):
        return I(self.t + 1, self.n_up + 1)

    def go_down(self):
        return I(self.t + 1, self.n_up)


In [36]:
class Params(namedtuple('ParamsBase', [
    # company parameters
    'initial_valuation',
    'strike_price',
    'annual_volatility',
    'annual_growth',
    # numerics parameters
    'annual_timesteps',
    # offer parameters
    'vesting_period',  # years
    'ownership_fraction',
    'opportunity_cost',  # annual $ lost from not working at bigco
])):

    @property
    def ts_volatility(self):
        return self.annual_volatility / np.sqrt(self.annual_timesteps)

    @property
    def ts_growth(self):
        return self.annual_growth ** (1/self.annual_timesteps)

    @property
    def ts_sim_interval(self):
        return self.vesting_period * self.annual_timesteps
    
    @property
    def ts_vesting_interval(self):
        return self.vesting_period * self.annual_timesteps
    
    @property
    def ts_gain(self):
        """Amount the valuation goes up per one 'up' timestep"""
        # Source: http://www.maths.usyd.edu.au/u/UG/SM/MATH3075/r/Slides_7_Binomial_Market_Model.pdf
        return np.exp(self.ts_volatility)

    @property
    def ts_loss(self):
        return 1/self.ts_gain
    
    @property
    def ts_opportunity_cost(self):
        return self.opportunity_cost / self.annual_timesteps

    @property
    def p_growth(self):
        """Probability that each timestep is an 'up' timestep"""
        # Source: http://www.maths.usyd.edu.au/u/UG/SM/MATH3075/r/Slides_7_Binomial_Market_Model.pdf
        return (self.ts_growth - self.ts_loss) / (self.ts_gain - self.ts_loss)

    @property
    def ts_vesting_increments(self):
        cliff = self.annual_timesteps
        end = self.ts_vesting_interval
        month_len = self.annual_timesteps // 12
        return [0] + list(range(cliff, end + 1, month_len))

    @property
    def p_loss(self):
        return 1 - self.p_growth

In [48]:
# coherence checks:
params = Params(2e7, 5e6, 1.0, 1.08, 24, 4, 0.006, 60000)
# computed expected growth should be 1.08
print((params.p_growth * params.ts_gain + (1 - params.p_growth) * params.ts_loss) ** params.annual_timesteps)
# computed annual volatility should be 0.54
print(np.sqrt(((params.p_growth * params.ts_gain ** 2 + (1 - params.p_growth) * params.ts_loss ** 2)
               - params.ts_growth ** 2) * params.annual_timesteps))
print(params.ts_vesting_increments)

1.08
1.00322105667
[0, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96]


In [5]:
print('Super naive option value:', (params.initial_valuation - params.strike_price) * params.ownership_fraction)
print('Total cost:', params.opportunity_cost * params.vesting_period)

Super naive option value: 90000.0
Total cost: 240000


In [42]:
CACHE_INVAL_TIME = datetime.datetime.now()
def memoize(fn):
    # inval the cache bc we're probably redefining a function
    global CACHE_INVAL_TIME
    CACHE_INVAL_TIME = datetime.datetime.now()
    fn.cache = {}
    fn.cache_inval_time = datetime.datetime.now()
    def inner(*args):
        if fn.cache_inval_time < CACHE_INVAL_TIME:
            fn.cache = {}
            fn.cache_inval_time = datetime.datetime.now()
        if args not in fn.cache:
            fn.cache[args] = fn(*args)
        return fn.cache[args]
    return inner

In [52]:
def get_valuation(pm, i):
    return pm.initial_valuation * pm.ts_gain ** (i.n_up - i.n_down)


@memoize
def get_payoff(pm, i, t_quit, t_vesting_end):
    """Get the expected payoff from state i, if you quit at time t_quit"""
    if i.t == pm.ts_sim_interval:
        # pretend we exercise at the end of the vesting period
        ts_vesting = pm.ts_vesting_interval if t_vesting_end is None else t_vesting_end
        ts_working = pm.ts_sim_interval if t_quit is None else t_quit
        # TODO(ben): apply time discounting the opportunity cost
        cost = ts_working * pm.ts_opportunity_cost
        # TODO(ben): extrapolate to IPO
        full_payoff = max(get_valuation(pm, i) - pm.strike_price, 0)
        return full_payoff * pm.ownership_fraction * (ts_vesting / pm.ts_vesting_interval) - cost
    elif t_quit is not None:
        # we already quit, so just run through to the end
        return (pm.p_growth * get_payoff(pm, i.go_up(), t_quit, t_vesting_end)
                + pm.p_loss * get_payoff(pm, i.go_down(), t_quit, t_vesting_end))
    else:
        return max(get_payoff_if_quit(pm, i), get_payoff_if_stay(pm, i))

def get_payoff_if_quit(pm, i):
    """Get the expected payoff from state i, if you quit exactly then"""
    # vesting ends at the last "vesting increment"
    t_vesting_end = max(t for t in pm.ts_vesting_increments if t <= i.t)
    return get_payoff(pm, i, i.t, t_vesting_end)

def get_payoff_if_stay(pm, i):
    """Get the expected payoff from state i, if you quit exactly then"""
    return (pm.p_growth * get_payoff(pm, i.go_up(), None, None)
            + pm.p_loss * get_payoff(pm, i.go_down(), None, None))

print(get_payoff(params, I(0, 0), None, None))

29114.450761


In [53]:
@memoize
def get_naive_payoff(pm, i):
    """Get the expected payoff from state i if you can't quit"""
    if i.t == pm.vesting_period * pm.annual_timesteps:
        cost = pm.vesting_period * pm.opportunity_cost
        payoff = max(get_valuation(pm, i) - pm.strike_price, 0)
        return payoff * pm.ownership_fraction - cost
    else:
        return (pm.p_growth * get_naive_payoff(pm, i.go_up())
                + pm.p_loss * get_naive_payoff(pm, i.go_down()))

print(get_naive_payoff(params, I(0, 0)))

-95175.1203478
